In [2]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mode
import random
from scipy.ndimage import rotate, zoom
from sklearn.preprocessing import PowerTransformer
import pickle
from tqdm import tqdm
from multiprocessing import Pool
import gc

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from keras import models, layers, losses, optimizers, Model
from keras.utils import plot_model

2023-11-07 01:46:02.711372: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-07 01:46:02.711414: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-07 01:46:02.711465: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
def get_feature_vectors():
    with open("feature_vecs.txt", "r") as f:
        f = f.read().split("\n\n\n")
        noun_dict = {}
        for noun_vec in f:
            split = noun_vec.split("\n\n")
            noun = split[0].split()[2][:-1]
            
            vec = split[1]
            vec = [item.strip().replace("(", "").replace(")", "") for item in vec.split(",\n")]
            vec = [(item.split()[:-1], item.split()[-1]) for item in vec]

            vec = sorted(vec, key=lambda x: x[0])
            vec = {" ".join(item[0]): float(item[1]) for item in vec}

            noun_dict[noun] = vec

    return {k: [noun_dict[k][k1] for k1 in noun_dict[k]] for k in noun_dict}

feature_vectors = get_feature_vectors()

{'bear': [0.041,
  0.069,
  0.024,
  0.085,
  0.158,
  0.054,
  0.013,
  0.031,
  0.143,
  0.018,
  0.041,
  0.0,
  0.054,
  0.167,
  0.19,
  0.011,
  0.079,
  0.018,
  0.181,
  0.52,
  0.52,
  0.018,
  0.486,
  0.07,
  0.214],
 'cat': [0.028,
  0.033,
  0.146,
  0.053,
  0.435,
  0.068,
  0.041,
  0.031,
  0.208,
  0.041,
  0.075,
  0.002,
  0.088,
  0.069,
  0.175,
  0.031,
  0.052,
  0.022,
  0.303,
  0.592,
  0.449,
  0.163,
  0.027,
  0.075,
  0.048],
 'cow': [0.014,
  0.027,
  0.039,
  0.042,
  0.421,
  0.043,
  0.367,
  0.048,
  0.061,
  0.013,
  0.023,
  0.0,
  0.045,
  0.115,
  0.056,
  0.009,
  0.067,
  0.04,
  0.261,
  0.74,
  0.205,
  0.073,
  0.027,
  0.08,
  0.025],
 'dog': [0.031,
  0.06,
  0.072,
  0.042,
  0.742,
  0.078,
  0.015,
  0.02,
  0.075,
  0.014,
  0.022,
  0.0,
  0.079,
  0.097,
  0.105,
  0.011,
  0.054,
  0.004,
  0.301,
  0.294,
  0.46,
  0.062,
  0.018,
  0.023,
  0.049],
 'horse': [0.012,
  0.039,
  0.01,
  0.05,
  0.082,
  0.034,
  0.023,
  0.017,
  0.

In [4]:
NUM_CLASSES = 60

pickles = [pickle.load(open(f"pickles/{i}.pkl", "rb")) for i in range(1)]
targets = set([pickles[0][i][1] for i in range(len(pickles[0]))])

targets = list(targets)[:NUM_CLASSES]
targets = {target: [[0 if i != j else 1 for i in range(NUM_CLASSES)]] for j, target in enumerate(targets)}

pickles = [item for sublist in pickles for item in sublist if item[1] in targets]
pickles = sorted(pickles, key=lambda x: x[1])

trains, tests = [pickles[i] for i in range(len(pickles)) if i % 6 != 0], [pickles[i] for i in range(len(pickles)) if i % 6 == 0]

train_x = np.array([train[1] for train in trains])
train_x = [feature_vectors[item] for item in train_x]
train_x = np.array(train_x)
train_y = np.array([train[0] for train in trains])


test_x = [test[0] for test in tests]
test_y = [test[1] for test in tests]

print(len(pickles), len(train_x), len(train_y), len(test_x), len(test_y))

del pickles, trains, tests

359 299 299 60 60


In [5]:
def get_sample(train=True):
    x, y = ((train_x, train_y) if train else (test_x, test_y))
    
    i = random.randint(0, len(x) - 1)

    target, scan = x[i], y[i]
    # scan = np.expand_dims(scan, -1)
    
    return scan, target

In [6]:
def plot_scan(scan):
    w = 5
    fig, ax = plt.subplots(w, w, constrained_layout=True)
    fig.dpi = 100
    bg_color = (225 / 255, 216 / 255, 226 / 255)
    fig.set_facecolor(bg_color)

    for j in range(w * w):
        ax[(j - j % w) // w, j % w].imshow(scan[j % scan.shape[0]], vmin=-1, vmax=1, cmap="twilight")
        ax[(j - j % w) // w, j % w].set_xticks([])
        ax[(j - j % w) // w, j % w].set_yticks([])
        plt.setp(ax[(j - j % w) // w, j % w].spines.values(), color=bg_color)

    plt.show()

In [7]:
def get_batch(batch_size=32, train=True):
    
    samples = list(Pool(processes=8).imap(get_sample, [train] * batch_size))
    batch_x = [sample[1] for sample in samples]
    batch_y = [sample[0] for sample in samples]

    return np.array(batch_x), np.array(batch_y)

a, b = get_batch(16)
a.shape, b.shape

((16, 25), (16, 23, 61, 51))

In [8]:
class BasisSum(Model):
    def __init__(self):
        super().__init__()
        self.basis = tf.Variable(tf.zeros((23, 61, 51, 25)))

    @tf.function(reduce_retracing=True)
    def call(self, x):
        x = tf.matmul(self.basis, tf.reshape(x, (-1, 1, 1, 1, 25)), transpose_b=True)
        x = tf.squeeze(x, axis=-1)
        return x

In [16]:
BATCH_SIZE = 128

model = BasisSum()

opt = optimizers.SGD(learning_rate=1)
loss = losses.MeanAbsoluteError()

train_x, train_y = tf.convert_to_tensor(train_x), tf.convert_to_tensor(train_y)

print(train_x.shape)

with tf.GradientTape() as tape:
    pred = model(train_x)
    batchloss = loss(train_y, pred)
    print(batchloss)

(299, 25)
tf.Tensor(0.7708701, shape=(), dtype=float32)
